# PORTAL DA TRANSPARENCIA

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Inicializa o driver do Chrome
driver = webdriver.Chrome()

# Abre a URL de busca
driver.get("https://portaldatransparencia.gov.br/pessoa-juridica/busca/lista?pagina=1&tamanhoPagina=10&")

# Define tempo máximo de espera
wait = WebDriverWait(driver, 10)

# Termo de busca
entidade = 'COBAP'

# Aguarda o campo de busca e envia o termo
campo_busca = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="termo"]')))
campo_busca.clear()
campo_busca.send_keys(entidade)
campo_busca.send_keys(Keys.ENTER)

# Aguarda os resultados carregarem
time.sleep(5)



# Lista para armazenar os links
lista_hrefs = []

while True:
    try:
        # Aguarda os links da página atual carregarem
        #wait.until(EC.presence_of_all_elements_located((
        #    By.XPATH, '//div[@id="lista-resultado__itens"]//a[starts-with(@href, "/documento/acordao-completo")]'
        #)))

        # Coleta os links da página atual
        #links = driver.find_elements(
        #    By.XPATH,
        #    '//div[@id="lista-resultado__itens"]//a[starts-with(@href, "/documento/acordao-completo")]'
        #)

        # Localiza o elemento pelo XPath
        # Pega todos os links dentro do span com id="resultados"
        links = driver.find_elements(By.XPATH, '//span[@id="resultados"]//a[@class="link-busca-nome"]')

        # Itera e imprime os hrefs
        for link in links:
            href = link.get_attribute('href')
            #print(href)
            if href:
                full_href = href
                lista_hrefs.append(full_href)
                print(full_href)


        # Aguarda o botão "Próxima" estar clicável
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Próxima")]'))
        )

        # Verifica se está desativado (exemplo: por alguma classe ou condição no href)
        # Aqui o "href" ainda está como javascript:void(0);, então talvez a classe mude ou o botão não responda
        if "disabled" in next_button.get_attribute("class").lower():
            print("Última página.")
            break

        # Clica no botão "Próxima"
        next_button.click()

    except:
        print("Botão 'Próxima' não encontrado ou não clicável.")
        break

# Exporta os dados para CSV
df = pd.DataFrame(lista_hrefs, columns=["href"])
df['ENTIDADE'] = entidade  
df.to_csv("acordaos_contag.csv", index=False, encoding="utf-8")
print("Arquivo 'acordaos_contag.csv' salvo com sucesso.")

# Encerra o navegador
driver.quit()




https://portaldatransparencia.gov.br/busca/pessoa-juridica/07329386000129-cobap-comercio-e-beneficiamento-de-artefatos-de-papel-s-a
https://portaldatransparencia.gov.br/busca/pessoa-juridica/04510647000114-baterias-cobap-ltda
https://portaldatransparencia.gov.br/busca/pessoa-juridica/73963522000162-julio-cleber-tannus-de-souza
https://portaldatransparencia.gov.br/busca/pessoa-juridica/91340141000109-confederacao-brasileira-de-aposentados-pensionistas-e-idosos
https://portaldatransparencia.gov.br/busca/pessoa-juridica/11535891000105-t-e-s-comercio-varejista-de-pecas-e-acessorios-para-veiculos-ltda
Botão 'Próxima' não encontrado ou não clicável.
Arquivo 'acordaos_contag.csv' salvo com sucesso.


In [4]:
df

,href,ENTIDADE
0,https://portaldatransparencia.gov.br/busca/pes...,COBAP
1,https://portaldatransparencia.gov.br/busca/pes...,COBAP
2,https://portaldatransparencia.gov.br/busca/pes...,COBAP
3,https://portaldatransparencia.gov.br/busca/pes...,COBAP
4,https://portaldatransparencia.gov.br/busca/pes...,COBAP


# TCU

In [66]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

# Carrega os links do CSV
df = pd.read_csv("acordaos_contag.csv")


# Supondo que seu DataFrame seja df e a coluna se chame 'CONTAG'
valores_unicos = df['ENTIDADE'].unique()

# Converte o array numpy em lista de strings
valores_unicos_str = valores_unicos.astype(str).tolist()

entidade = valores_unicos_str[0]  # pega o primeiro (e único) item da lista
print(entidade)      # 'CONTAG'


# Inicia o navegador
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

lista_de_dados = []

dados = []

for url in df["href"]:
    print(url)
    driver.get(url)

    wait = WebDriverWait(driver, 10)

    # Aguarda os elementos que contêm "ACÓRDÃO" ou "DECISÃO"
    elemento = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@id='conteudo_numero_acordao']//a"))
    )

    # Extrai o texto
    texto = elemento.text.strip()
    match = re.match(r"^[A-ZÇÁÂÉÍÓÚÃÕÜ]+", texto)
    if match:
        tipo = match.group()
        #print(">>", match.group())



    elemento_data = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//h3[contains(text(),'Data da sessão')]/following-sibling::div//p"))
    )

    data_sessao = elemento_data.text.strip()







    lista_de_dados.append({
        "ENTIDADADE":entidade,
        "TIPO": match.group(),
        "TITULO": texto,
        "DATA": data_sessao
    })


    # Quando terminar todas as iterações, cria o DataFrame
    df = pd.DataFrame(lista_de_dados)




    #print(texto)

    # Extrai e imprime os textos encontrados
    #for t in titulos:
    #    print(t)
        #texto = t.text.strip()
        #if texto:
        #    print(">>", texto)

# Fecha o navegador
driver.quit()


CONTAG


https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/0
https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/1


In [67]:
df

,ENTIDADADE,TIPO,TITULO,DATA
0,CONTAG,ACÓRDÃO,ACÓRDÃO DE RELAÇÃO 2052/2025 - PLENÁRIO,03/09/2025
1,CONTAG,ACÓRDÃO,ACÓRDÃO 1115/2024 - PLENÁRIO,05/06/2024


In [61]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Inicializa o driver do Chrome
driver = webdriver.Chrome()

# Abre a URL de busca
driver.get("https://pesquisa.apps.tcu.gov.br/resultado/todas-bases/*?ts=1757626103727&pb=acordao-completo")

# Define tempo máximo de espera
wait = WebDriverWait(driver, 10)

# Termo de busca
entidade = 'CONTAG'

# Aguarda o campo de busca e envia o termo
campo_busca = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="termo"]')))
campo_busca.clear()
campo_busca.send_keys(entidade)
campo_busca.send_keys(Keys.ENTER)

# Aguarda os resultados carregarem
time.sleep(5)

# Lista para armazenar os links
lista_hrefs = []

while True:
    try:
        # Aguarda os links da página atual carregarem
        wait.until(EC.presence_of_all_elements_located((
            By.XPATH, '//div[@id="lista-resultado__itens"]//a[starts-with(@href, "/documento/acordao-completo")]'
        )))

        # Coleta os links da página atual
        links = driver.find_elements(
            By.XPATH,
            '//div[@id="lista-resultado__itens"]//a[starts-with(@href, "/documento/acordao-completo")]'
        )

        for link in links:
            href = link.get_attribute("href")
            if href:
                full_href = href
                lista_hrefs.append(full_href)
                print(full_href)

        # Tenta encontrar o botão de próxima página
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Próxima página"]'))
        )

        # Verifica se está desativado
        if next_button.get_attribute("disabled"):
            print("Última página.")
            break

        # Força o clique via JavaScript para evitar interceptação
        driver.execute_script("arguments[0].click();", next_button)

        # Aguarda leve carregamento
        time.sleep(2)

    except Exception as e:
        print("Erro ou fim da paginação:", e)
        break

# Exporta os dados para CSV
df = pd.DataFrame(lista_hrefs, columns=["href"])
df['ENTIDADE'] = entidade  
df.to_csv("acordaos_contag.csv", index=False, encoding="utf-8")
print("Arquivo 'acordaos_contag.csv' salvo com sucesso.")

# Encerra o navegador
driver.quit()



https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/0
https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/1
https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/2
https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/3
https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/4
https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/5
https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%252C%2520NUMACORDAOINT%2520desc/6
https://pesquisa.apps.tcu.gov.br/documento/acordao-completo/CONTAG/%2520/DTRELEVANCIA%2520desc%25

# CGU

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Inicializa o driver do Chrome
driver = webdriver.Chrome()

# Abre a página de busca por pessoa jurídica
driver.get("https://eaud.cgu.gov.br/relatorios/pesquisa?colunaOrdenacao=dataPublicacao&direcaoOrdenacao=DESC&tamanhoPagina=31&offset=0")

# Espera até o campo de busca estar disponível
wait = WebDriverWait(driver, 5)




# --- OU ---
# (opção 2) Clica no botão de buscar (caso queira clicar em vez de dar ENTER)
# botao_buscar = driver.find_element(By.XPATH, '//button[@type="submit"]')
# botao_buscar.click()

time.sleep(3)


entidade = 'contag'

# Localiza o elemento com id="resultados"
try:
    # Localiza o input dentro do div pela classe
    input_element = driver.find_element(By.XPATH, "//div[@class='br-input input-highlight input-pesquisa-relatorio']//input")

    # Interage com o campo
    input_element.clear()
    input_element.send_keys(entidade)

    # Pressiona ENTER para buscar (opção 1)
    input_element.send_keys(Keys.ENTER)

    time.sleep(5)

    relatorios = driver.find_elements(
        By.XPATH,
        "//div[contains(@class, 'div-relatorio')]"
    )

    lista_de_dados = []
    for relatorio in relatorios:
        try:
            titulo_element = relatorio.find_element(By.XPATH, ".//p[contains(@class, 'titulo-pesquisa-relatorio')]")
            subtitulo_element = relatorio.find_element(By.XPATH, ".//small[contains(@class, 'subtitulo-pesquisa-relatorio')]")
            
            titulo = titulo_element.text
            subtitulo = subtitulo_element.text
            onclick_attr = titulo_element.get_attribute("onclick")

            import re
            match = re.search(r"relatorio/' \+ (\d+)", onclick_attr)
            numero = match.group(1) if match else "n/a"
            url = f"https://eaud.cgu.gov.br/relatorio/{numero}" if numero != "n/a" else "Link não encontrado"

            #print("Título:", titulo)
            #print("Subtítulo:", subtitulo)

            so_titulo = titulo.split("Título:")[-1].strip()
            #print("Link:", url)

            # Divide por " • "
            partes = subtitulo.split(" • ")

            dados = {}

            for parte in partes:
                chave, valor = parte.split(":", 1)
                dados[chave.strip()] = valor.strip()

            # Exibindo
            #print("Titulo:", so_titulo)
            #print("Link:", url)
            #print("Relatório:", dados.get("Relatório"))
            #print("Publicado:", dados.get("Publicado"))
            #print("Tipo de Serviço:", dados.get("Tipo de Serviço"))
            #print("Localidade(s):", dados.get("Localidade(s)"))
            #print("Unidade(s) Auditada(s):", dados.get("Unidade(s) Auditada(s)"))

            lista_de_dados.append({
                "ENTIDADADES":entidade,
                "NUMERO_DE_RELATORIO": dados.get("Relatório"),
                "TITULO": so_titulo,
                "RESUMO": '-',
                "TIPO_DE_SERVICO": dados.get("Tipo de Serviço"),
                "DATA_DE_PUBLICADO": dados.get("Publicado"),
                "LOCALIDADES(S)": dados.get("Localidade(s)"),
                "UNIDADE(S) AUDITADA(s)": dados.get("Unidade(s) Auditada(s)"),
                "LINK": url,
                "LINK+DE_PDF": f'https://eaud.cgu.gov.br/relatorios/download/{numero}'
            })


            # Quando terminar todas as iterações, cria o DataFrame
            df = pd.DataFrame(lista_de_dados)



        except Exception as e:
            print("Erro ao processar relatório:", e)

    #for x in valores_lista_relatorios:
    #    print(x.text)
    #for x in valores_lista_relatorios.find_element(By.XPATH, "//div[2]"):
    #    print(x)
    #    print(valores_lista_relatorios)


    print("Elemento encontrado!")
except Exception as e:
    print("Elemento não encontrado:", e)


# Aguarda os resultados aparecerem
time.sleep(3)  # ou use wait.until(...) para algo mais confiável

# Fecha o navegador
driver.quit()

Elemento encontrado!


In [3]:
# !pip install selenium